TODO:
- Any binaries?
- Anti-correlations?

In [51]:
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table, join, vstack
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.io import ascii

from scipy.interpolate import interp1d
from scipy.stats import binned_statistic

import gala.coordinates as gc
import gala.dynamics as gd
from gala.units import galactic
from pyia import GaiaData

import corner

In [87]:
globclust[np.array(['M 4' in x['Name'] for x in globclust])]

Name,RA,DEC,D,Vlos,eVlos,PMRA,PMDEC,ePMRA,ePMDEC,corrPM,radius,Nmember
str21,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
NGC 6121 (M 4),245.897,-26.526,2.2,71.0,0.1,-12.484,-18.983,0.005,0.004,0.023,20.0,18796


In [11]:
g = GaiaData('/Users/adrian/data/APOGEE_DR15beta/allStar-t9-l31c-58158-with-gaiadr2.fits')
apogee_c = g.get_skycoord(distance=False, 
                          radial_velocity=g.VHELIO_AVG*u.km/u.s)
len(g)

397129

In [32]:
m_h_mask = (g.M_H > -3) & (g.M_H < 0.8)

In [12]:
globclust = Table.read('/Users/adrian/data/Misc/Vasiliev-globclust.txt', format='ascii.fixed_width')
cs = coord.SkyCoord(ra=globclust['RA']*u.deg, 
                    dec=globclust['DEC']*u.deg,
                    distance=globclust['D']*u.kpc,
                    pm_ra_cosdec=globclust['PMRA']*u.mas/u.yr,
                    pm_dec=globclust['PMDEC']*u.mas/u.yr,
                    radial_velocity=globclust['Vlos']*u.km/u.s)

In [81]:
X = np.vstack((apogee_c.radial_velocity.value, 
               g.M_H,
               g.ALPHA_M,
               g.MG_FE - g.AL_FE)).T

names=['RV [{0:latex_inline}]'.format(u.km/u.s),
       '[M/H]',
       r'[$\alpha$/M]',
       '[Mg/Al]']

for i in range(len(globclust)):
    c = cs[i]
    row = globclust[i]
    name = row['Name']
    
    sep_mask = apogee_c.separation(c) < 1.5*u.deg
    pm_mask = np.sqrt((apogee_c.pm_ra_cosdec-c.pm_ra_cosdec)**2 + 
                      (apogee_c.pm_dec-c.pm_dec)**2) < 1.5*u.mas/u.yr
    rv_mask = np.abs(apogee_c.radial_velocity - c.radial_velocity) < 20*u.km/u.s
    rough_all_mask = sep_mask & pm_mask & rv_mask
    
    nearby_mask = ((apogee_c.separation(c) > 3*u.deg) & 
                   (apogee_c.separation(c) < 10*u.deg) &
                   (np.abs(g.MG_FE) < 3) & (np.abs(g.AL_FE) < 3))
    
    if (m_h_mask & rough_all_mask).sum() >= 8:
        
        mask = m_h_mask & sep_mask & pm_mask
        lims = [(c.radial_velocity.value - 150,
                 c.radial_velocity.value + 150),
                (-3, 1.), 
                (-1.5, 1.5),
                (-1.5, 1.5)]
        
        # fig = corner.corner(X[nearby_mask], range=lims, bins=64, zorder=-100, cmap='Blues',
        #                     plot_contours=False, plot_density=True, plot_datapoints=False)
        
        fig = corner.corner(X[mask], labels=names, range=lims, bins=64, 
                            data_kwargs=dict(color='k', marker='o', ms=3, alpha=0.65),
                            plot_contours=False, plot_density=False, plot_datapoints=True,
                            truths=[c.radial_velocity.value, np.nan, np.nan, np.nan],
                            truth_color='tab:green',
                            hist_kwargs=dict(density=True))
        
        if nearby_mask.sum() >= 8:
            fig = corner.corner(X[nearby_mask], range=lims, bins=64, color='tab:blue',
                                data_kwargs=dict(marker='o', ms=1, alpha=0.2),
                                plot_contours=False, plot_density=False, plot_datapoints=True,
                                hist_kwargs=dict(density=True),
                                fig=fig)
        
        filename = name.replace('(', '').replace(')', '').replace(' ', '_')
        fig.savefig('../plots/globular-clusters/{0}.png'.format(filename), dpi=250)
        plt.close(fig)
        
        print(row['Name'], sep_mask.sum(), (sep_mask&pm_mask).sum(), rough_all_mask.sum())

/Users/adrian/anaconda/lib/python3.6/site-packages/astropy/units/quantity.py:639: RuntimeWarning: invalid value encountered in less
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


NGC 362 264 33 30


Pal 1 774 101 36


NGC 1851 526 71 63


NGC 1904 (M 79) 382 39 27


...

NGC 6760 135 35 17


NGC 6809 (M 55) 448 101 98


NGC 6838 (M 71) 1094 113 63


NGC 7078 (M 15) 1408 232 154


NGC 7089 (M 2) 529 41 34
